In [7]:
from music21 import *
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Activation
import glob
from keras.utils import to_categorical

In [8]:
notes = []

for file in glob.glob("Jazz/*.mid"):
    print("START")
    midi = converter.parse(file)
    print("midi")
    print(midi)
    notes_to_parse = None

    parts = instrument.partitionByInstrument(midi)
    print("parts")
    #print(parts)

    if parts: # file has instrument parts
        print("file has parts")
        notes_to_parse = parts.parts[0].recurse()
        print(notes_to_parse)
    else: # file has notes in a flat structure
        print("file doesnt have parts")
        notes_to_parse = midi.flat.notes
    print("element for loop started")
    for element in notes_to_parse:
        #print(element)
        if isinstance(element, note.Note):
            #print(element.pitch)
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            #print(element.normalOrder)
            notes.append('.'.join(str(n) for n in element.normalOrder))

START
midi
<music21.stream.Score 0x7f37f3888048>
parts
file has parts
<music21.stream.iterator.RecursiveIterator for Part:None @:0>
element for loop started
START
midi
<music21.stream.Score 0x7f37f38d8da0>
parts
file has parts
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
element for loop started
START
midi
<music21.stream.Score 0x7f37f3c6ae80>
parts
file has parts
<music21.stream.iterator.RecursiveIterator for Part:Violin @:0>
element for loop started


In [9]:
sequence_length = 100

# get all pitch names
pitchnames = sorted(set(item for item in notes))

n_vocab=len(pitchnames)

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

print(note_to_int)

{'0': 0, '0.3': 1, '0.3.7': 2, '0.4.7': 3, '1': 4, '1.7': 5, '10': 6, '10.0.4': 7, '10.1.3': 8, '10.2': 9, '10.2.4': 10, '10.2.5': 11, '11': 12, '2': 13, '2.3.5': 14, '2.5': 15, '2.5.7': 16, '2.5.7.10': 17, '2.5.7.9.10': 18, '2.5.9': 19, '2.6.9': 20, '3.5.7.9': 21, '3.5.9': 22, '3.7': 23, '4': 24, '4.6.10': 25, '4.7.9.0': 26, '5': 27, '5.9.0': 28, '6': 29, '6.10.1': 30, '7': 31, '7.10.0': 32, '7.10.2': 33, '8.10.0.2.4': 34, '8.9': 35, '9': 36, '9.0.3': 37, '9.0.3.5': 38, '9.1.4': 39, '9.10.2.3': 40, '9.10.2.5': 41, '9.2.3': 42, 'A1': 43, 'A2': 44, 'A3': 45, 'A4': 46, 'B-2': 47, 'B-3': 48, 'B-4': 49, 'B1': 50, 'B4': 51, 'C#2': 52, 'C#3': 53, 'C#4': 54, 'C#5': 55, 'C4': 56, 'C5': 57, 'D1': 58, 'D2': 59, 'D4': 60, 'D5': 61, 'E-4': 62, 'E-5': 63, 'E1': 64, 'E2': 65, 'E3': 66, 'E4': 67, 'E5': 68, 'F#2': 69, 'F#4': 70, 'F2': 71, 'F3': 72, 'F4': 73, 'G#4': 74, 'G1': 75, 'G2': 76, 'G3': 77, 'G4': 78}


In [10]:
network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
print(n_vocab)

# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input/float(n_vocab)
#print(network_input)
network_output = to_categorical(network_output)
print(network_output)

79
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [11]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.2))
#model.add(LSTM(512, return_sequences=True))
#model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [12]:
model.fit(network_input, network_output, epochs=200, batch_size=64)

Epoch 1/200
192/632 [========>.....................] - ETA: 113s - loss: 5.0011

/home/charan/.local/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (2.389944). Check your callbacks.
  % delta_t_median)


256/632 [===========>..................] - ETA: 74s - loss: 4.9090 

/home/charan/.local/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (1.195596). Check your callbacks.
  % delta_t_median)


632/632 [==============================] - 55s - loss: 4.4919    
Epoch 2/200
632/632 [==============================] - 8s - loss: 3.9303     
Epoch 3/200
632/632 [==============================] - 8s - loss: 3.5264     
Epoch 4/200
632/632 [==============================] - 8s - loss: 3.3463     
Epoch 5/200
632/632 [==============================] - 8s - loss: 3.2809     
Epoch 6/200
632/632 [==============================] - 9s - loss: 3.2170     
Epoch 7/200
632/632 [==============================] - 9s - loss: 3.0966     
Epoch 8/200
632/632 [==============================] - 8s - loss: 2.9962     
Epoch 9/200
632/632 [==============================] - 10s - loss: 2.9567    
Epoch 10/200
632/632 [==============================] - 10s - loss: 2.8172    
Epoch 11/200
632/632 [==============================] - 10s - loss: 2.7550    
Epoch 12/200
632/632 [==============================] - 10s - loss: 2.7010    
Epoch 13/200
632/632 [==============================] - 10s - loss: 2.652

632/632 [==============================] - 9s - loss: 0.1374     
Epoch 105/200
632/632 [==============================] - 9s - loss: 0.0921     
Epoch 106/200
632/632 [==============================] - 11s - loss: 0.0648    
Epoch 107/200
632/632 [==============================] - 13s - loss: 0.0983    
Epoch 108/200
632/632 [==============================] - 12s - loss: 0.0660    
Epoch 109/200
632/632 [==============================] - 11s - loss: 0.0715    
Epoch 110/200
632/632 [==============================] - 11s - loss: 0.0730    
Epoch 111/200
632/632 [==============================] - 10s - loss: 0.0816    
Epoch 112/200
632/632 [==============================] - 10s - loss: 0.0623    
Epoch 113/200
632/632 [==============================] - 9s - loss: 0.0711     
Epoch 114/200
632/632 [==============================] - 9s - loss: 0.0773     
Epoch 115/200
632/632 [==============================] - 10s - loss: 0.0676    
Epoch 116/200
632/632 [==============================]

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               6553856   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 79)                20303     
_________________________________________________________________
activation_2 (Activation)    (None, 79)                0         
Total para

In [15]:
import random
start = numpy.random.randint(0, len(network_input)-1)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []
print(len(pattern))
# generate 500 notes
for note_index in range(500):
    #print(note_index)
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))

    prediction = model.predict(prediction_input, verbose=0)

    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output=numpy.append(prediction_output,result)
    #print(pattern)
    pattern=numpy.append(pattern,index/n_vocab)
    
    #print(len(pattern))
    pattern = pattern[1:]
    #print(pattern)

100


In [16]:
prediction_output

array(['6.10.1', 'C#2', 'B-2', 'B4', 'C#3', 'B4', 'B-2', '6.10.1', 'C#2',
       'B1', 'F#4', 'F#2', 'F#2', '6.10.1', 'C#3', 'B-2', 'B-4', 'C#3',
       'B-4', 'F#2', '6.10.1', 'B-2', 'C#5', 'D4', '2.6.9', 'D2', 'D2',
       '2.6.9', 'A1', 'F#2', 'G4', 'A2', 'G4', 'F#2', '2.6.9', 'A1', 'G1',
       'D4', 'D2', 'D2', '2.6.9', 'A2', 'F#2', 'F#4', 'A2', 'F#4', 'D2',
       '2.6.9', 'F#2', 'A4', 'A3', '9.1.4', 'A1', 'A1', '9.1.4', 'E1',
       'C#2', 'D4', 'E2', 'D4', 'C#2', '9.1.4', 'E1', 'D1', 'A3', 'A1',
       'A1', '9.1.4', 'E2', 'C#2', 'C#4', 'E2', 'C#4', 'A1', '9.1.4',
       'C#2', 'E4', '3.5.9', '2.6.9', '3.5.9', '4.6.10', '3.5.9',
       '4.6.10', '3.5.9', '9.0.3.5', 'D4', '2.6.9', 'F#4', 'D4', '2.6.9',
       'D4', '2.6.9', 'F#4', 'D4', '2.6.9', 'F#4', '6.10.1', 'B-4', 'F#4',
       '6.10.1', 'F#4', '6.10.1', 'B-4', 'F#4', '6.10.1', 'D4', '2.6.9',
       'F#4', 'D4', '2.6.9', 'D4', '2.6.9', 'F#4', 'D4', '2.6.9', 'F#4',
       '6.10.1', 'B-4', 'F#4', '6.10.1', 'F#4', '6.10.1', 'B

In [17]:
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model

for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [18]:
midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'